In [1]:
from torchvision import datasets, transforms
import numpy as np
import qiskit
from quantum_c2c import quantum_c2c

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from qiskit import transpile, assemble

In [2]:
%autosave 10

Autosaving every 10 seconds


# Data preparation

## Training data of autoencoder (pre-training)

In [3]:

# DataLoader
X_train_autoencoder = datasets.MNIST(root='./data', train=True, download=True,
                         transform=transforms.Compose([transforms.ToTensor()]))

# Leaving only labels 0 and 1 
idx = np.append(np.where(X_train_autoencoder.targets == 0)[0], 
                np.where(X_train_autoencoder.targets == 1)[0])

X_train_autoencoder.data = X_train_autoencoder.data[idx]
X_train_autoencoder.targets = X_train_autoencoder.targets[idx]



## Training data of main training

In [4]:

# Concentrating on the first 100 samples
n_samples = 100

X_train = datasets.MNIST(root='./data', train=True, download=True,
                         transform=transforms.Compose([transforms.ToTensor()]))

# Leaving only labels 0 and 1 
idx = np.append(np.where(X_train.targets == 0)[0][:n_samples], 
                np.where(X_train.targets == 1)[0][:n_samples])

X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

## Test data of main training

In [5]:


n_samples = 50

X_test = datasets.MNIST(root='./data', train=False, download=True,
                        transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_test.targets == 0)[0][:n_samples], 
                np.where(X_test.targets == 1)[0][:n_samples])

X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]


# Customized autoencoder_model and quantum_curcuit

## autoencoder_model

In [6]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args
    def forward(self, x):
        return x.view((x.size(0),)+self.shape)


class AutoEncoder(nn.Module):
    
    def __init__(self,input_shape,encoded_len):
        super(AutoEncoder, self).__init__()
        input_len=input_shape[-1]*input_shape[-2]
        # Encoder
        self.encoder = nn.Sequential(
            Reshape(-1,input_len),
            nn.Linear(input_len, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Tanh(),            
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, encoded_len),
            Reshape(encoded_len)
        )

        # Decoder
        self.decoder = nn.Sequential(
            Reshape(-1,encoded_len),
            nn.Linear(encoded_len, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 256),            
            nn.Tanh(),            
            nn.Linear(256, input_len),
            nn.Sigmoid(),
            Reshape(-1,input_shape[-1],input_shape[-2]),
        )
        
        self.original_shape = input_shape
    def forward(self, inputs):

        codes = self.encoder(inputs)
        decoded = self.decoder(codes)
        
        return codes, decoded

## quantum_curcuit

In [7]:

class QuantumCircuit:
    """ 
    This class provides a simple interface for interaction 
    with the quantum circuit 
    """
    
    def __init__(self, n_qubits, backend, shots):
        # --- Circuit definition ---
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        
        self._circuit.measure_all()
        # ---------------------------

        self.backend = backend
        self.shots = shots
    
    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        results = job.result().get_counts()
        
        if type(results)==list:
            expectations=[]
            for result in results:
                counts = np.array(list(result.values()))
                states = np.array(list(result.keys())).astype(float)
               
                # Compute probabilities for each state
                probabilities = counts / self.shots
                # Get state expectation
                expectation = np.sum(states * probabilities)
                
                expectations.append(expectation)
            
            return np.array(expectations)
        else:
            counts = np.array(list(results.values()))
            states = np.array(list(results.keys())).astype(float)
            
            # Compute probabilities for each state
            probabilities = counts / self.shots
            # Get state expectation
            expectation = np.sum(states * probabilities)
            
            return np.array([expectation])



class HybridFunction(Function):
    """ Hybrid quantum - classical function definition """
    
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward pass computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        expectation_z = ctx.quantum_circuit.run(input[0].tolist())
        result = torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)

        return result
        
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        
        try:
            
            gradients = np.array([gradients]).T
        except:
            gradients = np.array([t.numpy() for t in gradients] ).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None

class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self, backend, shots, shift,encoded_len):
        
        
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(encoded_len, backend, shots)
        self.shift = shift
        
    def forward(self, input):

        return HybridFunction.apply(input, self.quantum_circuit, self.shift)

# Quantum C2C training

In [8]:
encoded_len=2
epochs=10

In [9]:
trained_encoder_model,y_predicted=quantum_c2c(X_train_autoencoder,X_train,X_test,AutoEncoder(input_shape=X_train.data.shape,encoded_len=encoded_len),Hybrid(qiskit.Aer.get_backend('aer_simulator'), 100, np.pi / 2,encoded_len=encoded_len),'model',epochs=epochs,encoded_len=encoded_len)

Pre-training autoencoder
[1/10] Loss: 0.03550582751631737
[2/10] Loss: 0.03415246680378914
[3/10] Loss: 0.025988958775997162
[4/10] Loss: 0.021658243611454964
[5/10] Loss: 0.025266822427511215
[6/10] Loss: 0.025498468428850174
[7/10] Loss: 0.02511824667453766
[8/10] Loss: 0.02189912088215351
[9/10] Loss: 0.0218746867030859
[10/10] Loss: 0.02308611012995243
Training main model


/home/user/anaconda3/lib/python3.8/site-packages/qiskit/utils/deprecation.py:62: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  return func(*args, **kwargs)


Training [10%]	Loss: 0.5096
Training [20%]	Loss: 0.3784
Training [30%]	Loss: 0.3245
Training [40%]	Loss: 0.2887
Training [50%]	Loss: 0.2579
Training [60%]	Loss: 0.2263
Training [70%]	Loss: 0.2101
Training [80%]	Loss: 0.1944
Training [90%]	Loss: 0.1841
Training [100%]	Loss: 0.1742
Performance on test data:
	Loss: 0.1735
	Accuracy: 100.0%


In [10]:
import qiskit.tools.jupyter
%qiskit_version_table

/home/user/anaconda3/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
